# Criando tabelas e escrevendo dados

Neste último módulo, abordaremos tarefas de escrita em SQL, incluindo a criação de tabelas e dados, bem como a edição de dados existentes. A maior parte do uso de SQL é para leitura de dados, especialmente se você trabalha em uma função analítica ou de ciência de dados. Mas, ao trabalhar com a movimentação de dados de uma fonte para outra, você inevitavelmente precisará assumir a criação de dados em bancos de dados relacionais.

## Configurar

Primeiro, prepare-se. Baixe o arquivo de banco de dados SQLite `company_operations.db` e conecte-se a ele. Use também o `pandas` para exibir os resultados da nossa consulta SQL como um `DataFrame`.

In [ ]:
import sqlite3
import pandas as pd
import urllib.request

# baixe o banco de dados SQLite e conecte-se a ele
urllib.request.urlretrieve("https://github.com/thomasnield/anaconda_intro_to_sql/blob/main/company_operations.db?raw=true", "company_operations.db")
conn = sqlite3.connect('company_operations.db')

## Planejando um Banco de Dados

Se quiséssemos criar um novo banco de dados do zero, poderíamos declarar um arquivo de banco de dados inexistente em nosso caminho e o SQLite criará esse arquivo. No entanto, adicionaremos uma nova tabela ao arquivo de banco de dados `company_operations.db`.

Ao planejar um banco de dados ou adições a um banco de dados, considere cuidadosamente quais serão os relacionamentos entre as tabelas e quais campos precisam ser capturados. Certifique-se de que os dados sejam simples e normalizados, e que uma tabela não tente ser mais de uma coisa. Por exemplo, não queremos que uma tabela contenha as informações `CUSTOMER` e `CUSTOMER_ORDER`, pois essas são duas entidades separadas, então as armazenamos em tabelas separadas.

Observe que é fácil adicionar campos e tabelas a um banco de dados. No entanto, é muito mais difícil modificar e remover campos e tabelas quando há dependências entre pais e filhos.

Em nosso caso de uso, queremos criar uma tabela chamada `CUSTOMER_PAYMENT` que rastreia se os clientes pagaram parte ou a totalidade de um `CUSTOMER_ORDER`.

Os campos que precisamos rastrear incluem:

* `CUSTOMER_PAYMENT_ID` - Identificador da chave primária para cada registro
* `CUSTOMER_ORDER_ID` - Chave estrangeira que vincula cada pagamento a um pedido
* `RECEIVE_DATE` - Data de recebimento do pagamento
* `RECEIVE_AMOUNT` - Valor do pagamento recebido
* `MEMO` - Qualquer texto de memorando do cliente

## Atributos de campo em SQL

Ao criar uma tabela, é importante estar ciente dos atributos de campo de cada coluna. O SQLite é rápido e flexível com tipos de dados porque é tipado dinamicamente e não exige que uma determinada coluna contenha números ou texto, conforme especificado. Outras plataformas de dados o farão.

No SQLite, você pode consultar os tipos de dados de uma tabela existente usando um comando `PRAGMA` com a função `table_info()`. Abaixo, podemos ver qual é cada tipo de dado para cada coluna da tabela `CUSTOMER_ORDER`. Observe a coluna `type`, o indicador `notnull`, o valor padrão `dflt_value` e o indicador de chave primária `pk`. Todos esses são atributos que precisaremos definir ao criar nossas próprias tabelas.

In [ ]:
sql = """ 
PRAGMA table_info(CUSTOMER_ORDER);
"""

pd.read_sql(sql, conn)

Aqui está uma rápida análise de cada um desses atributos sobre um campo de tabela.

* **Tipo de Dado** (type) - O tipo de dado (por exemplo, numérico, texto, data) armazenado em uma determinada coluna.

* **Não Nulo** (notnull) - Indica se a coluna proíbe valores nulos.

* **Valor Padrão** (dflt_value) - O valor padrão a ser fornecido para uma coluna se nenhum valor for fornecido.

* **Chave Primária** (pk) - Indica se a coluna é a chave primária.

O tipo de dado de uma coluna pode ser o mais complexo para se entender completamente, pois cada plataforma SQL terá diferentes tipos de dados para escolher. O SQLite simplifica o armazenamento para apenas 5 tipos de dados: [NULL, Integer, Real, Text e Blob](https://www.sqlite.org/datatype3.html). Ao contrário de outras plataformas SQL, o SQLite não impõe consistência de tipo em uma coluna porque é tipado dinamicamente.

No entanto, o SQLite [nomeia os 5 tipos com afinidades de tipo](https://www.sqlite.org/datatype3.html) para que se assemelhem a tipos mais específicos em outras plataformas de banco de dados. Alguns desses tipos de afinidade incluem:

|TIPO|DESCRIÇÃO|
|---|---|
|`INTEGER` |Um inteiro discreto que pode ser negativo ou positivo, com variantes de tamanho como `TINYINT`, `SMALLINT`, `MEDIUMINT`, etc.|
|`CHAR(X)`|Um texto de largura fixa com comprimento de `X` caracteres. |
|`VARCHAR(X)`|Um texto de largura variável com no máximo `X` caracteres.|
|`FLOAT`|Um número de ponto flutuante|
|`DOUBLE`|Um número de ponto flutuante de precisão dupla|
|`DATE`|Um tipo de valor de data contendo um ano, mês e dia.|
|`TIME`|Um valor de tempo contendo hora, minuto, segundo e fração de segundo.|
|`DATETIME`|Mescla uma data e hora|

Observe que esta é apenas uma amostra dos tipos de dados com os quais trabalharemos neste notebook. Independentemente da plataforma de banco de dados que você utiliza, reserve um tempo para ler a documentação e se familiarizar com os tipos de dados para saber quais escolher.

## Criando e Removendo Tabelas

Para criar esta tabela, usamos o comando `CREATE TABLE`. Observe que, após o comando, colocamos o nome da tabela e, em seguida, dentro de parênteses, declaramos cada coluna, seu tipo de dado e seus comportamentos, separados por vírgulas.

A declaração da chave estrangeira é feita por último, apontando o `CUSTOMER_ORDER_ID` para sua chave primária correspondente na tabela pai `CUSTOMER_ORDER`.

In [ ]:
sql = """
CREATE TABLE CUSTOMER_PAYMENT (
  CUSTOMER_PAYMENT_ID INTEGER PRIMARY KEY NOT NULL,
  CUSTOMER_ORDER_ID INTEGER NOT NULL,
  RECEIVE_DATE DATE NOT NULL DEFAULT (date('now')), 
  RECEIVE_AMOUNT DOUBLE NOT NULL, 
  MEMO VARCHAR(100), 

  FOREIGN KEY (CUSTOMER_ORDER_ID) REFERENCES CUSTOMER_ORDER(CUSTOMER_ORDER_ID)
)
"""

conn.execute(sql)


Embora não haja registros nesta tabela, agora podemos executar uma consulta `SELECT` nela e ver que a tabela agora está lá.

In [ ]:
sql = """
SELECT * FROM CUSTOMER_PAYMENT
"""

pd.read_sql(sql, conn)


Se precisar excluir uma tabela, use o comando `DROP TABLE`. Isso só será permitido se nenhum registro filho apontar para esta tabela.

Se você executar este comando (sua execução está comentada propositalmente), certifique-se de executar a operação `CREATE TABLE` acima novamente antes de prosseguir.

In [ ]:
sql = """
DROP TABLE CUSTOMER_PAYMENT
"""

# conn.execute(sql)


## Escrevendo registros com INSERT

Para inserir um novo registro em uma tabela, use o comando `INSERT`. No mínimo, você precisa fornecer apenas os campos que não possuem valores padrão ou nulos. Nesse caso, precisamos fornecer apenas `CUSTOMER_ORDER_ID` e `RECEIVE_AMOUNT`. Observe que sinalizamos que esses são os campos que serão fornecidos entre parênteses e, em seguida, esses `VALUES` são fornecidos em um segundo conjunto de parênteses.

In [ ]:
sql = """
INSERT INTO CUSTOMER_PAYMENT (CUSTOMER_ORDER_ID, RECEIVE_AMOUNT)
VALUES (1, 550)
"""

conn.execute(sql)


Agora vamos selecionar na tabela `CUSTOMER_PAYMENT`.

In [ ]:
sql = """
SELECT * FROM CUSTOMER_PAYMENT
"""

pd.read_sql(sql, conn)


O `CUSTOMER_PAYMENT_ID` é um `INTEGER` e uma `PRIMARY KEY`, portanto, o SQLite atribuirá automaticamente um inteiro incremental caso nenhum seja fornecido, começando no número `1`. Conforme especificado anteriormente no comando `CREATE TABLE`, o `RECEIVE_DATE` assumirá como padrão a data de hoje, e o `MEMO` não possui a restrição `NOT NULL`, portanto, assume como padrão `NULL`.

Você também pode inserir vários registros em lote, fornecendo várias linhas depois de `VALUES`, separadas por vírgulas.

In [ ]:
sql = """
INSERT INTO CUSTOMER_PAYMENT (CUSTOMER_ORDER_ID, RECEIVE_DATE, RECEIVE_AMOUNT, MEMO) VALUES 
(2, '2020-05-01', 560, 'Thank you again!'), 
(4, '2020-05-05', 430, 'Payment 1 of 2'),
(4, '2020-05-10', 270, 'Payment 2 of 2')
"""

conn.execute(sql)


Execute uma consulta `SELECT` em `CUSTOMER_PAYMENT` e você verá os três registros adicionais adicionados.

In [ ]:
sql = """
SELECT * FROM CUSTOMER_PAYMENT
"""

pd.read_sql(sql, conn)


Observe que o Pandas possui uma função `to_sql()` conveniente para gravar rapidamente um `DataFrame` em uma determinada tabela SQL. Abaixo, anexamos mais três registros à nossa tabela, mas de um `DataFrame` do Pandas.

In [ ]:
df = pd.DataFrame(data=[
    (5, '2020-05-11', 610, 'Payment 1 of 3'),
    (5, '2020-05-15', 500, 'Payment 2 of 3'),
    (5, '2020-05-19', 450, 'Payment 3 of 3')
    ],
    columns=["CUSTOMER_ORDER_ID", "RECEIVE_DATE", "RECEIVE_AMOUNT", "MEMO"]
)

df.to_sql("CUSTOMER_PAYMENT",
          conn,
          if_exists="append", index=False)


> Nunca concatene manualmente uma string SQL. Se precisar construir uma string SQL, certifique-se de usar marcadores de interrogação "?" e injetá-la usando a API. Você pode [ler mais na documentação do SQLite para Python](https://docs.python.org/3/library/sqlite3.html).

## ATUALIZAR e EXCLUIR

Para atualizar um registro, use o comando `UPDATE` seguido da tabela de destino das alterações. Em seguida, use a palavra-chave `SET` para atribuir um ou mais campos (separados por vírgulas) a um novo valor.

Certifique-se de usar o comando `WHERE` se estiver direcionando apenas registros específicos. Caso contrário, ele atualizará todos os registros com essas alterações de atribuição. Abaixo, atualizamos o registro `CUSTOMER_PAYMENT` com `CUSTOMER_PAYMENT_ID` de `2` para que os valores `RECEIVE_AMOUNT` e `RECEIVE_DATE` sejam alterados.

In [ ]:
sql = """
UPDATE CUSTOMER_PAYMENT SET RECEIVE_AMOUNT = 580, RECEIVE_DATE = '2020-05-05'
WHERE CUSTOMER_PAYMENT_ID = 2
"""

conn.execute(sql)


Para excluir um ou mais registros, use o comando `DELETE` com a condição `WHERE` direcionada a esses registros. Digamos que queremos excluir registros onde nenhum `MEMO` foi fornecido. Uma boa prática é visualizar os registros que você deseja excluir com um `SELECT`.

In [ ]:
sql = """
SELECT * FROM CUSTOMER_PAYMENT
WHERE MEMO IS NULL 
"""

pd.read_sql(sql, conn)


Podemos então usar a condição `WHERE` com um comando `DELETE`, conforme demonstrado abaixo.

In [ ]:
sql = """
DELETE FROM CUSTOMER_PAYMENT
WHERE MEMO IS NULL 
"""

conn.execute(sql)


Veja as alterações executando esta consulta.

In [ ]:
sql = """
SELECT * FROM CUSTOMER_PAYMENT
"""

pd.read_sql(sql, conn)


## Transações

Ao fazer edições em um banco de dados, considere fortemente fazê-lo dentro de uma **transação**, que atua como um botão de retrocesso a partir do ponto em que a transação foi iniciada. Aliás, quando você começa a fazer operações de gravação como fizemos acima, uma transação que nunca foi finalizada já está aberta. Isso significa que precisamos executar `commit()` nas alterações que acabamos de fazer, caso contrário, elas serão perdidas no momento em que fecharmos a conexão com o banco de dados. Vamos confirmar essas alterações agora.

In [ ]:
conn.execute("COMMIT") 

O motivo para isso é que, se algo der errado, seja um erro, uma queda de energia, um erro de rede ou outros incidentes, precisamos restaurar o banco de dados ao seu último ponto de integridade.

Para iniciar uma transação manualmente, você a chama com SQL assim:

In [ ]:
conn.execute("BEGIN TRANSACTION") 

Agora estamos no modo de transação. Podemos usar um `try-except` para lidar com operações de gravação e, se algo der errado, podemos executar um `ROLLBACK`, caso contrário, executaremos um `COMMIT`. Abaixo está uma transação bem-sucedida em que dois registros são criados com sucesso.

In [ ]:
try: 

  conn.execute("INSERT INTO CUSTOMER_PAYMENT (CUSTOMER_ORDER_ID, RECEIVE_AMOUNT) VALUES (11, 720)")
  conn.execute("INSERT INTO CUSTOMER_PAYMENT (CUSTOMER_ORDER_ID, RECEIVE_AMOUNT) VALUES (12, 540)")
  
  conn.execute("COMMIT")

except: 
  print("FAILED! Rolling back")
  conn.execute("ROLLBACK")

Aqui está um exemplo que falha. Observe que o segundo `INSERT` não possui um valor para `RECEIVE_AMOUNT`.

In [ ]:
conn.execute("BEGIN TRANSACTION") 

try: 

  conn.execute("INSERT INTO CUSTOMER_PAYMENT (CUSTOMER_ORDER_ID, RECEIVE_AMOUNT) VALUES (15, 1020)")
  conn.execute("INSERT INTO CUSTOMER_PAYMENT (CUSTOMER_ORDER_ID, RECEIVE_AMOUNT) VALUES (17)")
  
  conn.execute("COMMIT")
  
except: 
  print("FAILED! Rolling back")
  conn.execute("ROLLBACK")

A transação falhará e será revertida, o que significa que o primeiro `INSERT` será revertido e, se você verificar a tabela `CUSTOMER_PAYMENT`, não deverá vê-lo lá.

In [ ]:
sql = """
SELECT * FROM CUSTOMER_PAYMENT
"""

pd.read_sql(sql, conn)


## Exercício

Complete o código abaixo para inserir os dois registros em uma transação em `CUSTOMER_PAYMENT`, que é confirmada em caso de sucesso e revertida em caso de falha. Forneça `CUSTOMER_ORDER_ID`, `RECEIVE_DATE` e `RECEIVE_AMOUNT`.

In [ ]:
conn.execute("?") 

try: 

  conn.execute("? INTO ? (?, ?, ?) VALUES (25, '2020-05-11',1090)")
  conn.execute("? INTO ? (?, ?, ?) VALUES (27, '2020-05-12',2070)")
  
  conn.execute("?")

except: 
  print("FAILED! Rolling back")
  conn.execute("?")

# display results 
pd.read_sql("SELECT * FROM CUSTOMER_PAYMENT", conn)


### RESPOSTA A BAIXO

|<br>
|<br>
|<br>
|<br>
|<br>
|<br>
|<br>
|<br>
|<br>
|<br>
|<br>
|<br>
|<br>
|<br>
|<br>
|<br>
|<br>
|<br>
|<br>
|<br>
|<br>
|<br>
|<br>
v 

In [ ]:
conn.execute("BEGIN TRANSACTION") 

try: 

  conn.execute("INSERT INTO CUSTOMER_PAYMENT (CUSTOMER_ORDER_ID, RECEIVE_DATE, RECEIVE_AMOUNT) VALUES (25, '2020-05-11',1090)")
  conn.execute("INSERT INTO CUSTOMER_PAYMENT (CUSTOMER_ORDER_ID, RECEIVE_DATE, RECEIVE_AMOUNT) VALUES (27, '2020-05-12',2070)")
  
  conn.execute("COMMIT")

except: 
  print("FAILED! Rolling back")
  conn.execute("ROLLBACK")

pd.read_sql("SELECT * FROM CUSTOMER_PAYMENT", conn)
